In [61]:
import pandas as pd
data = pd.read_csv('./prd1.csv')


In [74]:
import pandas as pd
import sqlite3
from tabulate import tabulate
# Load your CSV data into a DataFrame
data = pd.read_csv('./prd1.csv')

# Create an SQLite in-memory database (you can also use a file-based database)
conn = sqlite3.connect(':memory:')

# Store the DataFrame in the database as a table
data.to_sql('cric', conn, index=False)

# Run SQL queries on the data
query = """

SELECT
    Line,
    Length,
    Position,
    CRuns,
    CCount,
    TRuns,
    TCount,
    LLC.Count AS 'Line Length Combination',
    TotalShotFieldCount.Count AS 'Total Shot Field Count',
    cast (LLC.Count as float) / cast (TotalShotFieldCount.Count as float) as 'FR',

    (CAST(CRuns AS FLOAT) / CAST(CCount AS FLOAT)) / (CAST(TRuns AS FLOAT) / CAST(TCount AS FLOAT)) AS P1
FROM (
    SELECT
        Delivery_Line AS Line,
        Delivery_Length AS Length,
        Shot_Field_Position AS Position,
        SUM(Runs) AS 'CRuns',
        COUNT(*) AS 'CCount',
        (SELECT SUM(Runs) FROM cric) AS 'TRuns',
        (SELECT COUNT(*) FROM cric) AS 'TCount'
    FROM cric
    GROUP BY Delivery_Line, Delivery_Length, Shot_Field_Position
) AS subquery
JOIN (
    SELECT
        Delivery_Line AS LLC_Line,
        Delivery_Length AS LLC_Length,
        COUNT(*) AS Count
    FROM cric
    GROUP BY Delivery_Line, Delivery_Length
) AS LLC ON subquery.Line = LLC.LLC_Line AND subquery.Length = LLC.LLC_Length
JOIN (
    SELECT
        Shot_Field_Position AS ShotField,
        COUNT(*) AS Count
    FROM cric
    GROUP BY Shot_Field_Position
) AS TotalShotFieldCount ON subquery.Position = TotalShotFieldCount.ShotField;

"""

result = pd.read_sql_query(query, conn)
print(result)
# Convert the result to JSON
result_json = result.to_json()
table = tabulate(result, headers='keys', tablefmt='pretty', showindex=False)
result.to_csv('h.csv', index=False)
# Save the JSON data to a file

# Close the database connection
conn.close()


     Line       Length      Position  CRuns  CCount  TRuns  TCount  \
0     Leg   FullLength  BehindSquare      1       1   1310     929   
1     Leg   FullLength     CowCorner      0       1   1310     929   
2     Leg   FullLength   DeepFineLeg      2       1   1310     929   
3     Leg   FullLength       FineLeg      0       1   1310     929   
4     Leg   FullLength        LongOn      0       1   1310     929   
..    ...          ...           ...    ...     ...    ...     ...   
151  Wide   GoodLength   StraightHit      2       2   1310     929   
152  Wide   GoodLength      ThirdMan     32      10   1310     929   
153  Wide  ShortLength     ExraCover      1       1   1310     929   
154  Wide  ShortLength   StraightHit      2       2   1310     929   
155  Wide  ShortLength      ThirdMan      4       1   1310     929   

     Line Length Combination  Total Shot Field Count        FR        P1  
0                          6                       9  0.666667  0.709160  
1        